In [43]:
:opt no-lint

# Chapter 26 Monad Transformers
## 26.1 Monad Transformers
- various monad transformer types and instances.
- ordering and wrapping of monad transformer stacks.

## 26.2 MaybeT
The `Monad` instance works as follow:
- when the value is a `Nothing` it returns `Nothing`
- otherwise it extract the value and returns the bind of the second monad.

## 26.3 EitherT

In [44]:
newtype EitherT e m a = EitherT { runEitherT :: m (Either e a) }

### Exercises: EitherT
1. Write the `Functor` instance for `EitherT`:

In [45]:
instance Functor m => Functor (EitherT e m) where
  fmap f (EitherT x) = EitherT $ (fmap . fmap) f x

2. Write the `Applicative` instance for `EitherT`:

In [46]:
instance Applicative m => Applicative (EitherT e m) where
  pure = EitherT . pure . pure
  (EitherT f) <*> (EitherT a) = EitherT $ (<*>) <$> f <*> a

3. Write the `Monad` instance for `EitherT`:

In [47]:
instance Monad m => Monad (EitherT e m) where
  return = pure
  (EitherT ma) >>= f = EitherT $ do
    v <- ma
    case v of
      Left e -> pure $ Left e
      Right x -> runEitherT $ f x

4. Write the `swapEitherT` helper function for `EitherT`

In [48]:
swapEitherT :: (Functor m) => EitherT e m a -> EitherT a m e
swapEitherT (EitherT x) = EitherT $ swapEither <$> x

swapEither :: Either e a -> Either a e
swapEither (Left e) = Right e
swapEither (Right x) = Left x

5. Write the transformer variant of the either catamorphism:

In [49]:
eitherT :: Monad m => (a -> m c) -> (b -> m c)-> EitherT a m b -> m c
eitherT fl fr (EitherT v) = v >>= either fl fr

## 26.4 ReaderT
The `Monad` instance:
- evaluate the input value by applying the `r`
- evaluate the second monad with the result
- returns a function from `r` to the new monadic value

## 26.5 StateT

In [50]:
newtype StateT' s m a = StateT' { runStateT' :: s -> m (a,s) }

#### Exercises: StateT
1. Implement the `Functor` instance:

In [51]:
{-# LANGUAGE InstanceSigs #-}

import Data.Bifunctor

instance (Functor m) => Functor (StateT' s m) where
  fmap :: (a -> b) -> StateT' s m a -> StateT' s m b
  fmap f (StateT' sm) = StateT' $ (fmap . fmap) f' sm
    where f' (x, s) = (f x, s)

2. Implement the `Applicative` instance:

In [52]:
instance (Monad m) => Applicative (StateT' s m) where
  pure :: a -> StateT' s m a
  pure x = StateT' $ \s -> pure (x, s)

  (<*>) :: StateT' s m (a -> b) -> StateT' s m a -> StateT' s m b
  StateT' sf <*> StateT' sx = StateT' $ \s -> do
    (f, s') <- sf s
    (x, s'') <- sx s'
    pure (f x, s'')

We needed the `Monad` constraint because eache step of the computation depends on a previous one.

3. implement the `Monad` instance:

In [53]:
instance (Monad m) => Monad (StateT' s m) where
  return :: a -> StateT' s m a
  return = pure

  (>>=) :: StateT' s m a -> (a -> StateT' s m b) -> StateT' s m b
  StateT' v >>= f = StateT' $ \s -> do
    (x, s') <- v s
    runStateT' (f x) s'

## 26.6 Types you probably don’t want to use
- `Writer` or `WriterT`, as its lazyness in not suitable for logging long running applications.
- `ListT`, for performance reasons. Use a streaming library instead.

## 26.7 An ordinary type from a transformer
The _non-trasformer_ variant can be recovered as follow:
```haskell
type MyIdentity a = IdentityT Identity a
type Maybe a      = MaybeT Identity a
type Either e a   = EitherT e Identity a
type Reader r a   = ReaderT e Identity a
type State s a    = StateT s Identity a
```
Sometimes, the libraries only provide the _trasformer_ variant of a type.
## 26.8 Lexically inner is structurally outer
A series of monad transformers in a type will begin with the innermost type, structurally speaking.
Removing layers one-by-one:

In [54]:
import Control.Monad.Trans.Maybe  (MaybeT(..), runMaybeT)
import Control.Monad.Trans.Except (ExceptT(..), runExceptT)
import Control.Monad.Trans.Reader (ReaderT(..), runReaderT)

embedded :: MaybeT (ExceptT String (ReaderT () IO)) Int
embedded = return 1

maybeUnwrap :: ExceptT String (ReaderT () IO) (Maybe Int)
maybeUnwrap = runMaybeT embedded

eitherUnwrap :: ReaderT () IO (Either String (Maybe Int))
eitherUnwrap = runExceptT maybeUnwrap

readerUnwrap :: () -> IO (Either String (Maybe Int))
readerUnwrap = runReaderT eitherUnwrap

#### Exercise: Wrap it up
Turn `readerUnwrap` from the previous example back into embedded through the use of the data constructors for each transformer.

In [55]:
embedded' :: MaybeT (ExceptT String (ReaderT () IO)) Int
embedded' = MaybeT $ ExceptT $ ReaderT $ pure . const (Right (Just 1))

## 26.9 MonadTrans
`fmap`, `liftA` and `liftM` allow to lift a function respectively inside a `Functor`, `Applicative` and `Monad` structure.
In order to lift inside a monad transformer we need the `MonadTrans` class, which offers the `lift` functions.
We need to lift through all the layers of the monad transformer.
Lifting means embedding an expression in a larger context by adding structure that doesn’t do anything.
The general pattern with `MonadTrans` instances is that you’re usually going to lift the injection of the
known structure (e.g.: with `MaybeT`, the known structure is `Maybe`) over some `Monad`. Injection of structure usually means `return` (with `MaybeT`, as we need a `Maybe` structure, we use `Just`)
#### Exercises: Lift more

In [56]:
import Control.Monad.Trans.Class (MonadTrans, lift)
import Control.Monad             (liftM)

1. `EitherT`:

In [57]:
instance MonadTrans (EitherT e) where
  lift = EitherT . fmap Right

2. `StateT`:

In [58]:
instance MonadTrans (StateT' s) where
  lift v = StateT' $ \s -> do
    x <- v
    pure (x, s) 

#### Prolific lifting is the failure mode
In order to keep the `lift` implementation clean, it is advisable to newtype the `Monad` stack and abstracting away the representation.
## 26.10 MonadIO, aka zoom-zoom
`MonadIO` is an alternative way  to lift an action over additional structure. Rather than lifting through one layer at a time, `MonadIO` keeps lifting the `IO` action until it is lifted over all structure embedded in the outermost `IO` type.

In [59]:
import Control.Monad.IO.Class (MonadIO, liftIO)

#### Exercises: Some instances
1. `MaybeT`

In [60]:
{-# LANGUAGE GeneralizedNewtypeDeriving #-}
{-# LANGUAGE DeriveFunctor #-}
{-# LANGUAGE InstanceSigs #-}

newtype MaybeT' m a = MaybeT' { runMaybeT' :: m (Maybe a) } deriving Functor

instance Applicative m => Applicative (MaybeT' m) where
  pure = MaybeT' . pure . pure
  MaybeT' mmab <*> MaybeT' mma = MaybeT' $ (<*>) <$> mmab <*> mma

instance (Monad m) => Monad (MaybeT' m) where
  return = pure
  (MaybeT' mma) >>= f = MaybeT' $ do
    v <- mma
    case v of
      Nothing -> pure Nothing
      Just x  -> runMaybeT' $ f x

instance MonadTrans MaybeT' where
  lift :: Monad m => m a -> MaybeT' m a
  lift = MaybeT' . fmap Just

instance (MonadIO m) => MonadIO (MaybeT' m) where
  liftIO :: IO a -> MaybeT' m a
  liftIO = lift . liftIO

2. `ReaderT`

In [61]:
newtype ReaderT' r m a = ReaderT' { runReaderT' :: r -> m a }

instance (Functor m) => Functor (ReaderT' r m) where
  fmap f (ReaderT' rma) = ReaderT' $ (fmap . fmap) f rma

instance (Applicative m) => Applicative (ReaderT' r m) where
  pure = ReaderT' . pure . pure
  ReaderT' rmab <*> ReaderT' rma = ReaderT' $ (<*>) <$> rmab <*> rma

instance (Monad m) => Monad (ReaderT' r m) where
  return = pure
  (ReaderT' rma) >>= f = ReaderT' $ \r -> do
    a <- rma r
    runReaderT' (f a) r

instance MonadTrans (ReaderT' r) where
  lift :: Monad m => m a -> ReaderT' r m a
  lift = ReaderT' . const

instance (MonadIO m) => MonadIO (ReaderT' r m) where
  liftIO :: IO a -> ReaderT' r m a
  liftIO = lift . liftIO

3. `StateT`

In [62]:
instance (MonadIO m) => MonadIO (StateT' s m) where
  liftIO = lift . liftIO

## 26.11 Monad transformers in use
## 26.12 Monads do not commute
Monads in general do not commute.
## 26.13 Transform if you want to
When writing a program, start with the simplest thing, and eventually add structure later. Sometimes `liftIO` and plain `IO` is enough.
## 26.14 Chapter exercises
Write the code
1. `rDec` is a function that should get its argument in the context of `Reader` and return a value decremented by one:

In [63]:
import Control.Monad.Trans.Reader (Reader(..), runReader, ask)

rDec :: Num a => Reader a a
rDec = do
  x <- ask
  pure $ x - 1
  
runReader rDec 1
fmap (runReader rDec) [1..10]

0

[0,1,2,3,4,5,6,7,8,9]

2. Once you have an `rDec` that works, make it and any inner lambdas point-free, if that’s not already the case.

In [64]:
rDec' :: Num a => Reader a a
rDec' = ReaderT $ pure . subtract 1

runReader rDec' 1
fmap (runReader rDec') [1..10]

0

[0,1,2,3,4,5,6,7,8,9]

3. `rShow` is `show`, but in `Reader`:

In [65]:
import Data.Functor.Identity (Identity)

rShow :: Show a => ReaderT a Identity String
rShow = show <$> ask

runReaderT rShow 1
fmap (runReaderT rShow) [1..10]

Identity "1"

[Identity "1",Identity "2",Identity "3",Identity "4",Identity "5",Identity "6",Identity "7",Identity "8",Identity "9",Identity "10"]

4. Once you have an `rShow` that works, make it point-free.

In [66]:
rShow' :: Show a => ReaderT a Identity String
rShow' = ReaderT $ pure . show

runReaderT rShow 1
fmap (runReaderT rShow) [1..10]

Identity "1"

[Identity "1",Identity "2",Identity "3",Identity "4",Identity "5",Identity "6",Identity "7",Identity "8",Identity "9",Identity "10"]

5. `rPrintAndInc` will first print the input with a greeting, then return the input incremented by one:

In [67]:
rPrintAndInc :: (Num a, Show a) => ReaderT a IO a
rPrintAndInc = ReaderT $ \r -> do
  putStrLn $ "Hej: " ++ show r
  pure $ r + 1

runReaderT rPrintAndInc 1
traverse (runReaderT rPrintAndInc) [1..10]

Hej: 1
2

Hej: 1
Hej: 2
Hej: 3
Hej: 4
Hej: 5
Hej: 6
Hej: 7
Hej: 8
Hej: 9
Hej: 10
[2,3,4,5,6,7,8,9,10,11]

6. `sPrintIncAccum` first prints the input with a greeting, then “puts” the incremented input as the new state and returns the original input as a `String`:

In [68]:
import Control.Monad.Trans.State as ST (StateT(..), runStateT, get, put)

sPrintIncAccum :: (Num a, Show a) => StateT a IO String
sPrintIncAccum = do
  s <- get
  liftIO $ print $ "Hej: " ++ show s
  put $ s + 1
  pure $ show s

runStateT sPrintIncAccum 10
mapM (runStateT sPrintIncAccum) [1..5]

: 

#### Fix the code
The code won’t type check as written; fix it so that it does.

In [69]:
import Control.Monad (guard)

isValid :: String -> Bool
isValid v = '!' `elem` v

maybeExcite :: MaybeT IO String
maybeExcite = do
  v <- liftIO getLine
  guard $ isValid v
  pure v

doExcite :: IO ()
doExcite = do
  putStrLn "say something excite!"
  excite <- runMaybeT maybeExcite
  case excite of
    Nothing -> putStrLn "MOAR EXCITE"
    Just e -> putStrLn ("Good, was very excite: " ++ e)

#### Hit counter
We’re going to provide an initial scaffold of a scotty application that counts hits to specific URIs. It also prefixes the keys with a prefix defined on app initialization, retrieved via command line arguments:

In [70]:
{-# LANGUAGE OverloadedStrings #-}

import Control.Monad.Trans.Class
import Control.Monad.Trans.Reader
import Data.IORef
import qualified Data.Map as M
import Data.Maybe                     (fromMaybe)
import Data.Text.Lazy                 (Text)
import qualified Data.Text.Lazy as TL
import System.Environment             (getArgs)
import Web.Scotty.Trans         as WT (get)
import Web.Scotty.Trans               (ActionT(..), ScottyT(..), scottyT, param, html)

data Config = Config
    { counts :: IORef (M.Map Text Integer)
    , prefix :: Text
    }

type Scotty = ScottyT Text (ReaderT Config IO)
type Handler = ActionT Text (ReaderT Config IO)

bumpBoomp :: Text -> M.Map Text Integer -> (M.Map Text Integer, Integer)
bumpBoomp k m = let
    incremented = fromMaybe 0 (M.lookup k m) + 1
  in
    (M.insert k incremented m, incremented)

app :: Scotty ()
app = WT.get "/:key" $ do
  unprefixed <- param "key"
  config <- lift $ ReaderT pure
  let key = prefix config <> unprefixed 
  newInteger <- liftIO $ atomicModifyIORef (counts config) (\ref -> (ref, bumpBoomp key ref))
  html $ mconcat [ "<h1>Success! Count was: "
                 , TL.pack $ show newInteger
                 , "</h1>"
                 ]

main :: IO ()
main = do
  [prefixArg] <- getArgs
  counter <- newIORef M.empty
  let config = Config counter $ TL.pack prefixArg
      runR r = runReaderT r config
  scottyT 3000 runR app

#### Morra
1. Write the game Morra using `StateT` and `IO`. The state being
accumulated is the score of the player and the computer AI op-
ponent. To start, make the computer choose its play randomly.
On exit, report the scores for the player and the computer,
congratulating the winner.

In [71]:
{-# LANGUAGE MultiWayIf #-}

import Control.Monad.Trans.State as ST (modify)
import System.Random                   (randomRIO)

data Board = Board { player :: Int, opponent :: Int }
                   
instance Show Board where
  show (Board player opponent) = mconcat [ "Player: ", show player, "\n", "Opponent: ", show opponent]
             
type Morra = StateT Board IO

playerWins :: Board -> Board
playerWins (Board player opponent) = Board (player + 1) opponent

opponentWins :: Board -> Board
opponentWins (Board player opponent) = Board player (opponent + 1)

morra :: Morra ()
morra = do
  playerHand <- read <$> liftIO getLine
  computerHand <- liftIO $ randomRIO (0, 5)
  let hand = computerHand + playerHand
  playerGuess <- read <$> liftIO getLine
  computerGuess <- liftIO $ randomRIO (0, 10)
  if 
    | playerGuess == hand -> modify playerWins
    | computerGuess == hand -> modify opponentWins
  loop

loop :: Morra ()
loop = do
  liftIO $ putStrLn "Continue? (y or n)"
  choice <- liftIO getLine
  case choice of
    "y" -> morra  
    "n" -> do board <- ST.get
              liftIO $ print board
    _   -> do liftIO $ putStrLn "Invalid choice"
              loop

main :: Morra ()
main = morra

2. Add a human vs. human mode to the game with interstitial screens between input prompts so the players can change out of the hot seat without seeing the other player’s answer.

In [72]:
import System.Console.ANSI (clearScreen)

twoPlayersMorra :: Morra ()
twoPlayersMorra = do
  firstPlayerHand <- read <$> liftIO getLine
  firstPlayerGuess <- read <$> liftIO getLine
  liftIO clearScreen
  secondPlayerHand <- read <$> liftIO getLine
  secondPlayerGuess <- read <$> liftIO getLine
  let hand = secondPlayerHand + firstPlayerHand
  if 
    | firstPlayerGuess == hand -> ST.modify playerWins
    | secondPlayerGuess == hand -> ST.modify opponentWins
  loop

main :: Morra ()
main = do
  liftIO $ putStrLn "Number of players (1 or 2)? "
  players <- read <$> liftIO getLine
  case players of
    1 -> morra
    2 -> twoPlayersMorra
    _ -> do liftIO $ putStrLn "Invalid choice"
            main

3. Improve the computer AI slightly by making it remember 3-grams of the player’s behavior, adjusting its answer instead of deciding randomly when the player’s behavior matches a known behavior.

In [73]:
import qualified Data.Map as M (Map, empty, insert, lookup)
import Data.Maybe              (isJust, fromJust)
import Data.List               (isPrefixOf)

morra' :: Morra ()
morra' = do
  historyRef <- liftIO $ newIORef []
  playerHand <- read <$> liftIO getLine
  liftIO $ modifyIORef historyRef (playerHand:)
  playerGuess <- read <$> liftIO getLine
  history <- liftIO $ readIORef historyRef
  (computerHand, computerGuess) <- liftIO $ case predict history of
    Just (c:_) -> pure (0, c)
    Nothing -> do h <- randomRIO (0, 5)
                  g <- randomRIO (0, 10)
                  pure (h, g)
  let hand = computerHand + playerHand
  if 
    | playerGuess == hand -> ST.modify playerWins
    | computerGuess == hand -> ST.modify opponentWins
  loop

predict :: [Int] -> Maybe [Int]
predict xs = do
  pre <- takeMaybe 2 xs
  case followedWith pre xs of
    [] -> Nothing
    xxs -> Just xxs

followedWith :: [Int] -> [Int] -> [Int]
followedWith _ [] = []
followedWith pre (x:xs)
  | pre `isPrefixOf` xs = x : rest
  | otherwise = rest
  where
    rest = followedWith pre xs

takeMaybe :: Int -> [a] -> Maybe [a]
takeMaybe 0 _  = Just []
takeMaybe _ [] = Nothing
takeMaybe n (x:xs) = (x:) <$> takeMaybe (n - 1) xs

4. The 3-gram thing is pretty simple and dumb. Humans are bad at being random; they often have sub-patterns in their moves.

In [74]:
import qualified Data.Map as M (Map, empty, insert, lookup)
import Data.Maybe              (isJust, fromJust)
import Data.List               (isPrefixOf)

morra' :: Morra ()
morra' = do
  historyRef <- liftIO $ newIORef []
  playerHand <- read <$> liftIO getLine
  liftIO $ modifyIORef historyRef (playerHand:)
  playerGuess <- read <$> liftIO getLine
  history <- liftIO $ readIORef historyRef
  (computerHand, computerGuess) <- liftIO $ case predict history of
    Just (c:_) -> pure (0, c)
    Nothing -> do h <- randomRIO (0, 5)
                  g <- randomRIO (0, 10)
                  pure (h, g)
  let hand = computerHand + playerHand
  if 
    | playerGuess == hand -> ST.modify playerWins
    | computerGuess == hand -> ST.modify opponentWins
  loop

predict :: [Int] -> Maybe [Int]
predict xs = do
  pre <- takeMaybe 4 xs
  case followedWith pre xs of
    [] -> do
      pre' <- takeMaybe 3 xs
      case followedWith pre' xs of
        [] -> do
          pre'' <- takeMaybe 2 xs
          case followedWith pre'' xs of
            [] -> Nothing
            xxs -> Just xxs
        xxs -> Just xxs
    xxs -> Just xxs

followedWith :: [Int] -> [Int] -> [Int]
followedWith _ [] = []
followedWith pre (x:xs)
  | pre `isPrefixOf` xs = x : rest
  | otherwise = rest
  where
    rest = followedWith pre xs

takeMaybe :: Int -> [a] -> Maybe [a]
takeMaybe 0 _  = Just []
takeMaybe _ [] = Nothing
takeMaybe n (x:xs) = (x:) <$> takeMaybe (n - 1) xs

## 26.15 Definition
- *leak*: something that consumes a resource in a way that renders a program unusable or irrecoverable
- *memory leak*: consuming memory in a way that renders it unusable or unrecoverable by other programs or parts of a program

In [75]:
import Data.IORef                        
                                         
newtype Counter = Counter { x :: IORef Int }

i = 5

do
  iref <- newIORef i
  c' <- readIORef iref
  modifyIORef iref (+ i)
  writeIORef iref 6
  print(c')  
  c'' <- readIORef iref
  print(c'')  
  return (Counter iref)

5
6